# Analyze results for 3D CGAN
Feb 22, 2021

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns
from functools import reduce

In [2]:
from ipywidgets import *

In [3]:
%matplotlib widget

In [4]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/')
from modules_img_analysis import *

In [5]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/5_3d_cgan/1_main_code/')
import post_analysis_pandas as post


In [6]:
### Transformation functions for image pixel values
def f_transform(x,a):
    return 2.*x/(x + float(a)) - 1.

def f_invtransform(s,a):
    return float(a)*(1. + s)/(1. - s)

In [7]:
img_size=64
kappa=10
# img_size=128
# kappa=50

In [8]:
val_data_dict={'64':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing',
              '128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube'}

### Read validation data

In [9]:
# bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,100.5,5),np.arange(100.5,300.5,20),np.arange(300.5,1000.5,50),np.array([2000])]) #bin edges to use

bins=f_transform(bins,kappa)   ### scale to (-1,1)
# ### Extract validation data
sigma_lst=[0.5,0.65,0.8,1.1]
labels_lst=range(len(sigma_lst))
bkgnd_dict={}
num_bkgnd=100

for label in labels_lst:
#     fname=val_data_dict[str(img_size)]+'/norm_1_sig_{0}_train_val.npy'.format(sigma_lst[label])
    fname=val_data_dict[str(img_size)]+'/Om0.3_Sg%s_H70.0.npy'%(sigma_lst[label])
    print(fname)
    samples=np.load(fname,mmap_mode='r')[-num_bkgnd:][:,0,:,:]
    samples=f_transform(samples,kappa)
    dict_val=post.f_compute_hist_spect(samples,bins)
    bkgnd_dict[str(sigma_lst[label])]=dict_val
    del samples

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/Om0.3_Sg0.5_H70.0.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/Om0.3_Sg0.65_H70.0.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/Om0.3_Sg0.8_H70.0.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/Om0.3_Sg1.1_H70.0.npy


## Read data

In [10]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [11]:
dict1={'64':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/',
      '128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)


In [12]:
# parent_dir=u.result
parent_dir=dict1[str(img_size)]
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'202109*')]
n=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(n)

interactive(children=(Dropdown(description='x', options=('20210929_32023_cgan_bs32_nodes8_lr0.0001-fixed_fm10_…

In [13]:
result=n.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210929_32023_cgan_bs32_nodes8_lr0.0001-fixed_fm10_spec0.0_kappa10_goodrun


## Plot Losses

In [14]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)


In [15]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time,lr_d,lr_g
48755,48755.0,166.0,0.135684,0.161539,0.297222,3.669009,3.669009,0.0,-2.866034,0.0,-2.446710,NaN,NaN,3.417134,-4.185108,-3.623263,0.767353,0.0001,0.0001
48756,48756.0,166.0,0.173135,0.152502,0.325637,3.532031,3.532031,0.0,-2.305274,0.0,-3.047081,NaN,NaN,4.130112,-4.052407,-3.452937,0.766628,0.0001,0.0001
48757,48757.0,166.0,0.150930,0.150665,0.301596,3.867695,3.867695,0.0,-3.167958,0.0,-2.291162,NaN,NaN,3.997380,-4.016691,-3.816374,0.768648,0.0001,0.0001
48758,48758.0,166.0,0.072842,0.343400,0.416241,6.744773,6.744773,0.0,-2.936192,0.0,-3.093509,NaN,NaN,4.402650,-1.668949,-6.739353,0.766995,0.0001,0.0001
48759,48759.0,166.0,0.334530,0.163478,0.498008,3.188193,3.188193,0.0,-2.595462,0.0,-2.410016,NaN,NaN,1.352851,-5.456304,-3.085680,0.768401,0.0001,0.0001
48760,48760.0,166.0,0.174193,0.200386,0.374579,3.421082,3.421082,0.0,-3.171035,0.0,-2.304740,NaN,NaN,3.862727,-2.615827,-3.347498,0.760571,0.0001,0.0001
48761,48761.0,166.0,0.160490,0.103496,0.263986,4.679174,4.679174,0.0,-2.329460,0.0,-2.762429,NaN,NaN,4.607963,-2.625693,-4.641594,0.757476,0.0001,0.0001
48762,48762.0,166.0,0.197345,0.186689,0.384034,3.973563,3.973563,0.0,-1.641453,0.0,-2.106686,NaN,NaN,2.986954,-4.282259,-3.926453,0.784768,0.0001,0.0001
48763,48763.0,166.0,0.206353,0.186962,0.393315,3.661331,3.661331,0.0,-2.037332,0.0,-1.430610,NaN,NaN,3.132323,-3.393475,-3.584849,0.765341,0.0001,0.0001
48764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,-1.771052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [17]:

chi=df_metrics.quantile(q=0.2,axis=0)['hist_chi']
print(chi)
df_metrics[(df_metrics['hist_chi']<=chi)&(df_metrics.epoch>30)].sort_values(by=['hist_chi']).head(10)

-2.575203371047974


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time,lr_d,lr_g
17414,17414.0,59.0,0.203826,0.376485,0.580312,4.852725,4.852725,0.0,-2.224583,0.0,-3.858233,NaN,NaN,2.537645,-1.589133,-4.838430,0.773662,0.0001,0.0001
28127,28127.0,96.0,0.282202,0.335305,0.617507,3.474630,3.474630,0.0,-2.322382,0.0,-3.815233,NaN,NaN,2.395574,-1.737897,-3.427936,0.777550,0.0001,0.0001
35964,35964.0,123.0,0.185474,0.385766,0.571240,5.163580,5.163580,0.0,-3.162331,0.0,-3.814207,NaN,NaN,3.720729,-1.275906,-5.136268,0.818798,0.0001,0.0001
35223,35223.0,120.0,0.194000,0.678797,0.872797,8.429202,8.429202,0.0,-1.791970,0.0,-3.754769,NaN,NaN,4.587991,-0.513138,-8.428866,0.756769,0.0001,0.0001
19405,19405.0,66.0,0.169807,0.746542,0.916350,6.638426,6.638426,0.0,-2.291082,0.0,-3.743919,NaN,NaN,3.407357,-0.310569,-6.635488,0.767978,0.0001,0.0001
42571,42571.0,145.0,1.634430,0.189537,1.823967,2.677423,2.677423,0.0,-2.151110,0.0,-3.740661,NaN,NaN,-1.312361,-7.692892,-2.430788,0.761366,0.0001,0.0001
33195,33195.0,113.0,0.107103,0.215108,0.322211,3.985184,3.985184,0.0,-2.756656,0.0,-3.740547,NaN,NaN,3.905024,-2.933363,-3.936031,0.772897,0.0001,0.0001
29753,29753.0,101.0,0.246211,2.319983,2.566194,13.120228,13.120228,0.0,-2.450329,0.0,-3.730117,NaN,NaN,8.897449,2.207343,-13.120222,0.772808,0.0001,0.0001
46867,46867.0,160.0,0.173616,0.406669,0.580285,6.596627,6.596627,0.0,-2.475627,0.0,-3.723183,NaN,NaN,3.839986,-1.184574,-6.594589,0.761347,0.0001,0.0001
31092,31092.0,106.0,0.168406,0.202236,0.370641,4.494912,4.494912,0.0,-3.600881,0.0,-3.693198,NaN,NaN,2.919411,-2.133676,-4.434703,0.756582,0.0001,0.0001


In [18]:
# display(df_metrics.sort_values(by=['hist_chi']).head(8))
# display(df_metrics.sort_values(by=['spec_chi']).head(8))

<!-- ### Read validation data -->

## Read stored chi-squares for images

In [19]:
## Get sigma list from saved files
flist=glob.glob(result_dir+'/df_processed*')
sigma_lst=[i.split('/')[-1].split('df_processed_')[-1].split('.pkle')[0] for i in flist]
sigma_lst.sort() ### Sorting is important for labels to match !!

labels_lst=np.arange(len(sigma_lst))

In [20]:
sigma_lst,labels_lst

(['0.5', '0.8', '1.1'], array([0, 1, 2]))

In [21]:
### Create a merged dataframe

df_list=[]
for label in labels_lst:
    df=pd.read_pickle(result_dir+'/df_processed_{0}.pkle'.format(str(sigma_lst[label])))
    df[['epoch','step']]=df[['epoch','step']].astype(int)
    df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting
    df_list.append(df)

for i,df in enumerate(df_list):
    df1=df.add_suffix('_'+str(i))
    # renaming the columns to be joined on
    keys=['epoch','step','img_type','label']
    rename_cols_dict={key+'_'+str(i):key for key in keys}
#     print(rename_cols_dict)
    df1.rename(columns=rename_cols_dict,inplace=True) 
    df_list[i]=df1
    
df_merged=reduce(lambda x, y : pd.merge(x, y, on = ['step','epoch','img_type','label']), df_list)

### Get sum of all 4 classes for 3 types of chi-squares
for chi_type in ['chi_1','chi_spec1','chi_spec2','chi_1a','chi_1b','chi_1c']:
    keys=[chi_type+'_'+str(label) for label in labels_lst]
#     display(df_merged[keys].sum(axis=1))
    df_merged['sum_'+chi_type]=df_merged[keys].sum(axis=1)
del df_list


In [22]:
df

,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,...,chi_spec3,num_imgs,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_sdev,label
0,0,10,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,14.985242,0.008071,0.020782,15.014095,0.325778,3.171777,...,51.569362,16,0,0,"[0.7861630920526275, 0.5327410697049111, 0.144...","[0.00023411574725693658, 0.003797087163121879,...","[-0.50000000025, 0.16666666619444437, 0.416666...","[6465333.057591142, 574983.9690368405, 189307....","[446911.8437043899, 43424.88414363947, 16595.6...",0-10
1,0,20,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.146124,0.017447,0.027366,0.190937,0.001466,0.087123,...,52.980110,16,0,0,"[0.9841380124244421, 0.04654741286455657, 0.00...","[0.00016482049857467675, 0.0004119647970418054...","[-0.50000000025, 0.16666666619444437, 0.416666...","[609350.0943267788, 118894.34739798999, 114773...","[103774.33172629226, 8396.969154873608, 6405.2...",0-20
2,0,30,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.133879,0.017447,0.027366,0.178692,0.001297,0.002483,...,52.961153,16,0,0,"[0.9996032719841766, 0.0011901855466766359, 0....","[5.672157888088791e-06, 1.7016473652917514e-05...","[-0.50000000025, 0.16666666619444437, 0.416666...","[678483.7733361396, 161231.56539096247, 121876...","[86845.20596747621, 9352.691741877656, 5201.97...",0-30
3,0,40,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.135937,0.017447,0.027366,0.180750,0.001347,0.001906,...,53.158488,16,0,0,"[0.9999566083185818, 0.0001294612884305716, 1....","[4.030261400421607e-06, 1.1999201107271864e-05...","[-0.50000000025, 0.16666666619444437, 0.416666...","[297690.6052669516, 85911.17528850229, 68166.9...","[41195.3881118726, 4998.299979628256, 2835.871...",0-40
4,0,50,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.130834,0.017447,0.027366,0.175647,0.001238,0.023199,...,53.283768,16,0,0,"[0.9991962914694468, 0.0021386146541845803, 0....","[3.441119190712887e-05, 8.486487469152938e-05,...","[-0.50000000025, 0.16666666619444437, 0.416666...","[65020.40459536581, 22303.54927343514, 31648.2...","[4802.247374103448, 547.70923036979, 1461.7078...",0-50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4885,167,48860,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.002195,0.000839,1.272095,1.275130,0.000284,12.248601,...,48.011481,16,3,0,"[0.9887041415685363, 0.019727574375597825, 0.0...","[0.0008170676201667529, 0.0012539619150170778,...","[-0.50000000025, 0.16666666619444437, 0.416666...","[3838466.744412362, 1825766.9587807704, 992283...","[1110833.4603513782, 601814.1887561084, 321371...",167-48860
4886,167,48870,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.005727,0.000502,0.496945,0.503174,0.000270,12.081485,...,49.176353,16,4,0,"[0.9899937681695067, 0.017513108193827287, 0.0...","[0.000747650745614828, 0.0011600534552009534, ...","[-0.50000000025, 0.16666666619444437, 0.416666...","[3393153.1801692676, 1651980.3143771102, 91557...","[971244.8000099667, 581222.149088626, 311093.3...",167-48870
4887,167,48880,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.001379,0.003096,0.670966,0.675440,0.000272,11.554721,...,45.396472,16,2,0,"[0.9866795024749996, 0.023453323104378938, 0.0...","[0.0010204130980844266, 0.0016040036751037766,...","[-0.50000000025, 0.16666666619444437, 0.416666...","[4934061.629416336, 2310342.302594087, 1237813...","[1472080.1383694396, 787584.1966892328, 407689...",167-48880
4888,167,48890,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.012055,0.002402,0.208665,0.223122,0.000140,6.513516,...,48.919883,16,1,0,"[0.9902543999190917, 0.017646794641939377, 0.0...","[0.0007650633187474562, 0.0011985912030517234,...","[-0.50000000025, 0.16666666619444437, 0.416666...","[3412733.784969243, 1697883.3970175157, 946913...","[1084255.5947617383, 565645.0873657909, 317065...",167-48890


### Slice best steps

In [23]:
def f_slice_merged_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',label='all',params_lst=[0,1,2],head=10,epoch_range=[0,None],use_sum=True,display_flag=False):
    ''' View dataframe after slicing
    '''

    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ######### Apply cutoff to keep reasonable chi1 and chispec1
    #### Add chi-square columns to use
    chi_cols=[]
    if use_sum: ## Add sum chi-square columns
        for j in ['chi_1','chi_spec1','chi_1a','chi_1b','chi_1c']: chi_cols.append('sum_'+j)
        
    if label=='all': ### Add chi-squares for all labels
        for j in ['chi_1','chi_spec1','chi_1a','chi_1b','chi_1c']:
            for idx,i in enumerate(params_lst): chi_cols.append(j+'_'+str(idx))
    else: ## Add chi-square for specific label
        assert label in params_lst, "label %s is not in %s"%(label,params_lst)
        label_idx=params_lst.index(label)
        print(label_idx)
        for j in ['chi_1','chi_spec1','chi_spec2','chi_1a','chi_1b','chi_1c']: chi_cols.append(j+'_'+str(label_idx))
#     print(chi_cols)
    q_dict=dict(df_merged.quantile(q=cutoff,axis=0)[chi_cols])
    # print(q_dict)
    strg=['%s < %s'%(key,q_dict[key]) for key in chi_cols ]
    query=" & ".join(strg)
    # print(query)
    df=df.query(query)
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
    chis=[i for i in df_merged.columns if 'chi' in i]
    col_list=['label']+chis+['epoch','step']
    if (col_mode=='short'): 
        col_list=['label']+[i for i in df_merged.columns if i.startswith('sum')]
        col_list=['label']+chi_cols
    df2=df1.head(head)[col_list]
    
    if display_flag: display(df2) # Display df
    
    return df2

# f_slice_merged_df(df_merged,cutoff=0.3,sort_col='sum_chi_1',label=0.65,params_lst=[0.5,0.65,0.8,1.1],use_sum=True,head=2000,display_flag=False,epoch_range=[7,None])

In [24]:
cols_to_sort=np.unique([i for i in df_merged.columns for j in ['chi_1_','chi_spec1_'] if ((i.startswith(j)) or (i.startswith('sum')))])

w=interactive(f_slice_merged_df,df=fixed(df_merged),
cutoff=widgets.FloatSlider(value=0.3, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
use_sum=widgets.Checkbox(value=True),
label=ToggleButtons(options=['all']+sigma_lst), params_lst=fixed(sigma_lst),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=cols_to_sort
)
display(w)

interactive(children=(FloatSlider(value=0.3, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [25]:
df_sliced=w.result
[int(i.split('-')[1]) for i in df_sliced.label.values]

# df_sliced

[]

In [26]:
best_step=[]
df_test=df_merged.copy()
df_test=df_merged[((df_merged.epoch>50) & (df_merged.epoch<5000))&(df_merged.epoch>0)]
cut_off=1.0

best_step.append(f_slice_merged_df(df_test,cutoff=cut_off,sort_col='sum_chi_1',label='all',use_sum=True,head=4,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)
best_step.append(f_slice_merged_df(df_test,cutoff=cut_off,sort_col='sum_chi_spec1',label='all',use_sum=True,head=8,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)
best_step.append(f_slice_merged_df(df_test,cutoff=cut_off,sort_col='sum_chi_spec2',label='all',use_sum=True,head=2,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)
best_step.append(f_slice_merged_df(df_test,cutoff=cut_off,sort_col='sum_chi_1b',label='all',use_sum=True,head=2,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)

# best_step.append([46669,34281])
best_step=np.unique([i for j in best_step for i in j])
print(best_step)
best_step

[20480 24040 31630 32750 35190 36800 39910 41130 42000 42770 43060 44810
 45160 46580 48560 48570]


array([20480, 24040, 31630, 32750, 35190, 36800, 39910, 41130, 42000,
       42770, 43060, 44810, 45160, 46580, 48560, 48570])

In [27]:
# best_step=[6176]
# best_step= [23985,24570,25155,25740,26325,26910,27495]
# best_step=[int(i.split('-')[1]) for i in df_sliced.label.values]
# best_step=np.arange(40130,40135).astype(int)

In [28]:
df_best=df_merged[df_merged.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(16, 64)
[(70, 20480), (82, 24040), (108, 31630), (112, 32750), (120, 35190), (126, 36800), (136, 39910), (140, 41130), (143, 42000), (146, 42770), (147, 43060), (153, 44810), (154, 45160), (159, 46580), (166, 48560), (166, 48570)]


In [29]:
col_list=['label']+[i for i in df_merged.columns if i.startswith('sum')]
df_best[col_list]

,label,sum_chi_1,sum_chi_spec1,sum_chi_spec2,sum_chi_1a,sum_chi_1b,sum_chi_1c
2047,70-20480,0.197240,7.691202,1.271502,0.071960,0.009549,0.115731
2403,82-24040,0.507277,1.095802,0.759712,0.014497,0.001298,0.491482
3162,108-31630,0.875630,1.097346,0.173816,0.015172,0.001389,0.859069
3274,112-32750,0.191205,5.513821,0.858323,0.056914,0.005683,0.128608
3518,120-35190,1.725873,0.259218,0.016951,0.009227,0.007092,1.709555
3679,126-36800,1.500739,0.120711,0.035303,0.008600,0.013364,1.478775
3990,136-39910,2.614669,0.124693,0.051258,0.005496,0.011459,2.597714
4112,140-41130,1.081565,0.117889,0.042588,0.004818,0.010477,1.066269
4199,143-42000,0.151515,1.207708,0.041209,0.038483,0.008944,0.104088
4276,146-42770,4.013409,0.107110,0.033334,0.013824,0.050808,3.948777


### Interactive plot

In [30]:

def f_plot_hist_spec(df,param_labels,sigma_lst,steps_list,bkg_dict,plot_type,img_size):

    assert plot_type in ['hist','spec','grid','spec_relative'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative']:     fig=plt.figure(figsize=(6,6))
    for par_label in param_labels:
        df=df[df.step.isin(steps_list)]
#         print(df.shape)
        idx=sigma_lst.index(par_label)
        suffix='_%s'%(idx)
        dict_bkg=bkg_dict[str(par_label)]
        
        for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
            label=row.label+'_'+str(par_label)
            if plot_type=='hist':
                x1=row['hist_bin_centers'+suffix]
                y1=row['hist_val'+suffix]
                yerr1=row['hist_err'+suffix]
                x1=f_invtransform(x1,kappa)
                
                plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
            if plot_type=='spec':
                y2=row['spec_val'+suffix]
                yerr2=row['spec_sdev'+suffix]/np.sqrt(row['num_imgs'+suffix])
                x2=np.arange(len(y2))
                y2=x2**2*y2; yerr2=x2**2*yerr2 ## Plot k^2 P(y)
                plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
                plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

            if plot_type=='spec_relative':

                y2=row['spec_val'+suffix]
                yerr2=row['spec_sdev'+suffix]
                x2=np.arange(len(y2))

                ### Reference spectrum
                y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
                y=y2/y1
                ## Variance is sum of variance of both variables, since they are uncorrelated

                # delta_r= |r| * sqrt(delta_a/a)^2 +(\delta_b/b)^2) / \sqrt(N)
                yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row['num_imgs'+suffix])
                
                plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
                plt.plot(x2, y, marker=marker, linestyle=':',label=label)

            if plot_type=='grid':
                images=np.load(row['fname'+suffix])[:,0,:,:,0]
                print(images.shape)
                f_plot_grid(images[:8],cols=4,fig_size=(8,4))
            
        ### Plot reference data
        if plot_type=='hist':
            x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
            x=f_invtransform(x,kappa)
            plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
            plt.title('Pixel Intensity Histogram')
            plt.xscale('symlog',linthresh=50)
        
        if plot_type=='spec':
            y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
            x=np.arange(len(y))
            y=x**2*y; yerr=x**2*yerr ## Plot k^2 P(y)
            plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
            plt.title('Spectrum')
            plt.xlim(0,img_size/2)

        if plot_type=='spec_relative':
            plt.axhline(y=1.0,color='k',linestyle='-.')
            plt.title("Relative spectrum")
            plt.xlim(0,img_size/2)
            plt.ylim(0.5,2)

    if plot_type in ['hist','spec']:     
        plt.yscale('log')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    plt.show()

# f_plot_hist_spec(df_merged,[sigma_lst[-1]],sigma_lst,[best_step[0]],bkgnd_dict,'hist')

In [32]:
interact_manual(f_plot_hist_spec,df=fixed(df_best),
                param_labels=SelectMultiple(options=sigma_lst),sigma_lst=fixed(sigma_lst),
                steps_list=SelectMultiple(options=df_best.step.values),
                img_size=fixed(img_size),
                bkg_dict=fixed(bkgnd_dict),plot_type=ToggleButtons(options=['hist','spec','grid','spec_relative']))

interactive(children=(SelectMultiple(description='param_labels', options=('0.5', '0.8', '1.1'), value=()), Sel…

<function __main__.f_plot_hist_spec(df, param_labels, sigma_lst, steps_list, bkg_dict, plot_type, img_size)>

In [33]:
best_step

array([20480, 24040, 31630, 32750, 35190, 36800, 39910, 41130, 42000,
       42770, 43060, 44810, 45160, 46580, 48560, 48570])

In [ ]:
sigma=1.1
fname=val_data_dict[str(img_size)]+'/norm_1_sig_{0}_train_val.npy'.format(sigma)
print(fname)
a1=np.load(fname,mmap_mode='r')[-100:]
a1.shape

images=a1[:,0,:,:,0]
f_plot_grid(images[8:16],cols=4,fig_size=(8,4))


### Delete unwanted stored models
(Since deterministic runs aren't working well )

In [ ]:
# # fldr='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210119_134802_cgan_predict_0.65_m2/models'
# fldr=result_dir
# print(fldr)
# flist=glob.glob(fldr+'/models/checkpoint_*.tar')
# len(flist)

In [ ]:
# # Delete unwanted stored images
# for i in flist:
#     try:
#         step=int(i.split('/')[-1].split('_')[-1].split('.')[0])
#         if step not in best_step:
# #             print(step)
#             os.remove(i)
#             pass
#         else: 
#             print(step)
# #             print(i)
#     except Exception as e:
# #         print(e)
# #         print(i)
#         pass

In [ ]:
# best_step

In [ ]:
# ! du -hs  /global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.5_train_val.npy

In [ ]:
# fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/Om0.3_Sg0.5_H70.0.npy'
# np.load(fname,mmap_mode='r').shape